In [0]:
#직접 데이터프레임 만들기 : createDataFrame
valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
TableA = spark.createDataFrame(valuesA,['name','id'])

valuesB = [('Rutabaga',1),('Pirate',2),('Ninja',3),('Darth Vader',4)]
TableB = spark.createDataFrame(valuesB,['name','id'])

TableA.show()
TableB.show()

+---------+---+
 name| id|
+---------+---+
 Pirate| 1|
 Monkey| 2|
 Ninja| 3|
Spaghetti| 4|
+---------+---+

+-----------+---+
 name| id|
+-----------+---+
 Rutabaga| 1|
 Pirate| 2|
 Ninja| 3|
Darth Vader| 4|
+-----------+---+

In [0]:
ta = TableA.alias('ta')
tb = TableB.alias('tb')

In [0]:
#innerJoin
inner_join = ta.join(tb, ta.name == tb.name)
inner_join.show()

+------+---+------+---+
 name| id| name| id|
+------+---+------+---+
 Ninja| 3| Ninja| 3|
Pirate| 1|Pirate| 2|
+------+---+------+---+

In [0]:
#leftJoin
left_join = ta.join(tb, ta.name == tb.name, how = 'left')
left_join.show()

+---------+---+------+----+
 name| id| name| id|
+---------+---+------+----+
 Monkey| 2| null|null|
 Ninja| 3| Ninja| 3|
 Pirate| 1|Pirate| 2|
Spaghetti| 4| null|null|
+---------+---+------+----+

In [0]:
from pyspark.sql.functions import *
#null인거 뽑아내기 join -> filter
left_join = ta.join(tb, ta.name == tb.name,how = 'left')
left_join.show()
left_join.filter(col('tb.name').isNull()).show()

+---------+---+------+----+
 name| id| name| id|
+---------+---+------+----+
 Monkey| 2| null|null|
 Ninja| 3| Ninja| 3|
 Pirate| 1|Pirate| 2|
Spaghetti| 4| null|null|
+---------+---+------+----+

+---------+---+----+----+
 name| id|name| id|
+---------+---+----+----+
 Monkey| 2|null|null|
Spaghetti| 4|null|null|
+---------+---+----+----+

In [0]:
#rightJoin
left_join = ta.join(tb, ta.name == tb.name, how = 'right')
left_join.show()

+------+----+-----------+---+
 name| id| name| id|
+------+----+-----------+---+
 null|null|Darth Vader| 4|
 Ninja| 3| Ninja| 3|
Pirate| 1| Pirate| 2|
 null|null| Rutabaga| 1|
+------+----+-----------+---+

In [0]:
#fullOuterJoin
left_join = ta.join(tb, ta.name == tb.name, how = 'full')
left_join.show()

+---------+----+-----------+----+
 name| id| name| id|
+---------+----+-----------+----+
 null|null|Darth Vader| 4|
 Monkey| 2| null|null|
 Ninja| 3| Ninja| 3|
 Pirate| 1| Pirate| 2|
 null|null| Rutabaga| 1|
Spaghetti| 4| null|null|
+---------+----+-----------+----+

In [0]:
#데이터프레임을 만드는 다양한 방법
llist = [('bob','2015-01-31',4),('alice','2015-04-23',10)]
left = spark.createDataFrame(llist,['name','date','duration'])
right = spark.createDataFrame([('alice',100),('bob',23)], ['name','upload'])

df = left.join(right, left.name == right.name)
display(df)

name,date,duration,name,upload
alice,2015-04-23,10,alice,100
bob,2015-01-31,4,bob,23


In [0]:
df = left.join(right,['name'])
display(df)

name,date,duration,upload
alice,2015-04-23,10,100
bob,2015-01-31,4,23


In [0]:
#join의 다양한 컬럼지정 방법
inner_join = ta.join(tb, on = "name")
inner_join.show()

inner_join = ta.join(tb, on = "name", how = 'left')
inner_join.show()

inner_join = ta.join(tb, on = "name", how = 'right')
inner_join.show()

inner_join = ta.join(tb, on = "name", how = 'full')
inner_join.show()

#조인 컬럼 지정 3가지
#1, 직접
#2, [] 안에
#3, on = "..."

+------+---+---+
 name| id| id|
+------+---+---+
 Ninja| 3| 3|
Pirate| 1| 2|
+------+---+---+

+---------+---+----+
 name| id| id|
+---------+---+----+
 Monkey| 2|null|
 Ninja| 3| 3|
 Pirate| 1| 2|
Spaghetti| 4|null|
+---------+---+----+

+-----------+----+---+
 name| id| id|
+-----------+----+---+
Darth Vader|null| 4|
 Ninja| 3| 3|
 Pirate| 1| 2|
 Rutabaga|null| 1|
+-----------+----+---+

+-----------+----+----+
 name| id| id|
+-----------+----+----+
Darth Vader|null| 4|
 Monkey| 2|null|
 Ninja| 3| 3|
 Pirate| 1| 2|
 Rutabaga|null| 1|
 Spaghetti| 4|null|
+-----------+----+----+

In [0]:
df = left.join(right, ["name"])
df.show()
# on이나 []로 조인 컬럼 지정을 해 주면 중복되는 컬럼은 하나만 나옴

+-----+----------+--------+------+
 name| date|duration|upload|
+-----+----------+--------+------+
alice|2015-04-23| 10| 100|
 bob|2015-01-31| 4| 23|
+-----+----------+--------+------+

In [0]:
#parquet파일로 데이터프레임 만들어 join하기
peopleDF1 = spark.read.parquet("/FileStore/tables/test/people-10m.parquet")
peopleDF1.printSchema()
peopleDF2 = spark.read.parquet("/FileStore/tables/test/people-10m.parquet")
peopleDF2.printSchema()

root
-- id: integer (nullable = true)
-- firstName: string (nullable = true)
-- middleName: string (nullable = true)
-- lastName: string (nullable = true)
-- gender: string (nullable = true)
-- birthDate: timestamp (nullable = true)
-- ssn: string (nullable = true)
-- salary: integer (nullable = true)

root
-- id: integer (nullable = true)
-- firstName: string (nullable = true)
-- middleName: string (nullable = true)
-- lastName: string (nullable = true)
-- gender: string (nullable = true)
-- birthDate: timestamp (nullable = true)
-- ssn: string (nullable = true)
-- salary: integer (nullable = true)

In [0]:
#다양한 집계함수
avgSalaryDF = peopleDF1.select(avg("salary").alias("averageSalary"))
avgSalaryDF.show()

+-------------+
averageSalary|
+-------------+
72633.0076033|
+-------------+

In [0]:
roundedAvgSalaryDF = avgSalaryDF.select(round("averageSalary").alias("roundedAverageSalary"))
roundedAvgSalaryDF.show()

+--------------------+
roundedAverageSalary|
+--------------------+
 72633.0|
+--------------------+

In [0]:
salaryDF = peopleDF1.select(max('salary').alias('max'), min('salary').alias('min'), round(avg('salary')).alias('averageSalary'))
salaryDF.show()

+------+------+-------------+
 max| min|averageSalary|
+------+------+-------------+
180841|-26884| 72633.0|
+------+------+-------------+

In [0]:
print(peopleDF1.count())
peopleDFDistinctNamesDF = peopleDF1.select('firstName').distinct()
print(peopleDFDistinctNamesDF.count())
peopleDFDistinctNamesDF.show(1)

10000000
5113
+---------+
firstName|
+---------+
 Alayna|
+---------+
only showing top 1 row

In [0]:
#withColumnRenamed-> 칼럼명 바꾸기
ssaDFDistinctNamesDF = peopleDF.select("firstName").withColumnRenamed("firstName","transName").distinct()
print(ssaDFDistinctNamesDF.count())
ssaDFDistinctNamesDF.show(3)

5113
+---------+
transName|
+---------+
 Alayna|
 Melaine|
 Faye|
+---------+
only showing top 3 rows

In [0]:
#데이터프레임 join하기
joinDF = peopleDFDistinctNamesDF.join(ssaDFDistinctNamesDF,  peopleDFDistinctNamesDF.firstName == ssaDFDistinctNamesDF.transName)
print(joinDF.count())
joinDF = peopleDFDistinctNamesDF.join(ssaDFDistinctNamesDF, col("firstName") == col("transName"))
print(joinDF.count())

5113
5113

In [0]:
a = peopleDF.select("salary").show(3)
b = peopleDF.select(col("salary")).show(3)

a = peopleDF.select(avg("salary")).show(3)
b = peopleDF.select(avg(col("salary"))).show(3)

a = peopleDF.select(round("salary")).show(3)
b = peopleDF.select(round(col("salary"))).show(3)

a = peopleDF.select(min("salary")).show(3)
b = peopleDF.select(min(col("salary"))).show(3)

+------+
salary|
+------+
 56172|
 40203|
 53417|
+------+
only showing top 3 rows

+------+
salary|
+------+
 56172|
 40203|
 53417|
+------+
only showing top 3 rows

+-------------+
 avg(salary)|
+-------------+
72633.0076033|
+-------------+

+-------------+
 avg(salary)|
+-------------+
72633.0076033|
+-------------+

+----------------+
round(salary, 0)|
+----------------+
 56172|
 40203|
 53417|
+----------------+
only showing top 3 rows

+----------------+
round(salary, 0)|
+----------------+
 56172|
 40203|
 53417|
+----------------+
only showing top 3 rows

+-----------+
min(salary)|
+-----------+
 -26884|
+-----------+

+-----------+
min(salary)|
+-----------+
 -26884|
+-----------+

In [0]:
#abs, upper등은 string 으로 argument를 받지는 않음
peopleDF.select(abs(col('salary'))).show(3)

peopleDF.select(upper(col('firstName'))).show(3)

+-----------+
abs(salary)|
+-----------+
 56172|
 40203|
 53417|
+-----------+
only showing top 3 rows

+----------------+
upper(firstName)|
+----------------+
 PENNIE|
 AN|
 QUYEN|
+----------------+
only showing top 3 rows

In [0]:
peopleWithFixedSalariesDF = peopleDF.select("firstName","middleName","lastName","gender","birthDate","ssn", abs(col("salary")).alias("salary"))
peopleWithFixedSalariesDF.count()

Out[59]: 10000000

In [0]:
peopleWithFixedSalariesDF.show(3)

+---------+----------+----------+------+-------------------+-----------+------+
firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---------+----------+----------+------+-------------------+-----------+------+
 Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
+---------+----------+----------+------+-------------------+-----------+------+
only showing top 3 rows

In [0]:
#limit -> 이 안에 있는 수만큼 뽑아옴
peopleWithFixedSalariesDF = peopleWithFixedSalariesDF.select("*").orderBy("salary").limit(20)#****************************

In [0]:
peopleWithFixedSalariesDF.show()

+---------+----------+---------+------+-------------------+-----------+------+
firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---------+----------+---------+------+-------------------+-----------+------+
 Janene| Lili| Prinn| F|1986-04-06 05:00:00|923-50-6804| 2|
 Brook| Winifred| Durnell| F|1999-09-07 04:00:00|989-18-7019| 3|
 Garret| Garrett| Ashling| M|1959-10-19 04:00:00|918-39-6461| 4|
 Doloris| Domenica| Matic| F|1984-08-03 04:00:00|928-53-3688| 5|
 Derrick| Lyle| Newens| M|1963-04-23 05:00:00|925-95-9823| 6|
 Mamie| Jenine| Chafney| F|1988-08-10 04:00:00|958-51-1173| 8|
 George| Zane| Jeaneau| M|1995-08-03 04:00:00|974-23-6736| 15|
 Gwyneth| Tammara| Aleixo| F|1956-12-17 05:00:00|986-67-8193| 16|
 Nam| Elenora| Stobbs| F|1964-10-10 04:00:00|982-36-6725| 16|
Zachariah| Timmy| Cogman| M|1956-04-02 05:00:00|976-42-4527| 17|
 Alaina| Lucretia| Morcomb| F|1981-03-08 05:00:00|930-20-6877| 19|
 Darell| Dustin| Highnam| M|1989-05-25 04:00:00|920-11-8439| 19|
 Nova| Vanita| Uren| F|1973-05-27 04:00:00|946-25-5047| 20|
 Patricia| Janita| Ibert| F|1983-07-11 04:00:00|949-13-7834| 23|
 Nicholle| Raylene| Solon| F|1996-11-12 05:00:00|950-81-6244| 24|
Christene| Eveline|Pomphrett| F|1980-11-28 05:00:00|944-76-2840| 24|
 Tari| Lavelle| Larwood| F|1980-01-06 05:00:00|949-46-3791| 24|
 Karla| Lilliam| Behr| F|1991-08-21 04:00:00|959-19-4609| 28|
 Sergio| August| Blaycock| M|1970-05-17 04:00:00|951-81-8867| 30|
 Hai| Daren| Furlong| M|1957-08-15 04:00:00|971-36-3485| 31|
+---------+----------+---------+------+-------------------+-----------+------+

In [0]:
peopleWithFixedSalariesDF.select("*").orderBy("salary", ascending = False).limit(20).show(4)

+---------+----------+--------+------+-------------------+-----------+------+
firstName|middleName|lastName|gender| birthDate| ssn|salary|
+---------+----------+--------+------+-------------------+-----------+------+
 Hai| Daren| Furlong| M|1957-08-15 04:00:00|971-36-3485| 31|
 Sergio| August|Blaycock| M|1970-05-17 04:00:00|951-81-8867| 30|
 Karla| Lilliam| Behr| F|1991-08-21 04:00:00|959-19-4609| 28|
 Tari| Lavelle| Larwood| F|1980-01-06 05:00:00|949-46-3791| 24|
+---------+----------+--------+------+-------------------+-----------+------+
only showing top 4 rows